In [14]:
"""
MOVING AVERAGE CROSSOVER STRATEGY USING THE S&P 500. 
"""

import numpy as np
import pandas as pd
# import hvplot.pandas
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv


import stonk_pull

%matplotlib inline

In [15]:
#load alpaca api keys

api = stonk_pull.load()


C:\Users\costa\desktop\byte\rice\ru-hou-fin-pt-03-2020-u-c\Project3_DoH\chris\resources\stonk_pull.py:16: UserWarning: Make sure your api keys are renamed to fit this function or alter the function to fit your names
  warnings.warn('Make sure your api keys are renamed to fit this function or alter the function to fit your names')


In [16]:
#Pull the data from alphavantage
data = stonk_pull.data_pull(api)

In [17]:
#Clean the data
stock_data_df = stonk_pull.data_clean(data)
stock_data_df.tail()

,SPY Open,SPY Close,SPY Volume
date,,,
1999-11-05,138.6250,137.8750,7431500.0
1999-11-04,136.7500,136.5312,7907500.0
1999-11-03,136.0000,135.5000,7222300.0
1999-11-02,135.9687,134.5937,6516900.0
1999-11-01,136.5000,135.5625,4006500.0


In [18]:
#Sort earliest to latest.
stock_data_df.sort_index(inplace=True, ascending=True)
stock_data_df.tail()


,SPY Open,SPY Close,SPY Volume
date,,,
2020-08-17,337.94,337.91,34496002.0
2020-08-18,338.34,338.64,38733908.0
2020-08-19,339.05,337.23,68054244.0
2020-08-20,335.36,338.28,42207826.0
2020-08-21,337.92,339.48,53605220.0


In [82]:
#To set up the crossover strategy select the one column we need, "Close", and set to dataframe
signals_df = stock_data_df['SPY Close'].to_frame()

signals_df.head()


,SPY Close
date,
1999-11-01,135.5625
1999-11-02,134.5937
1999-11-03,135.5000
1999-11-04,136.5312
1999-11-05,137.8750


In [83]:
#Generate signals and put in dataframe

# Set the short window and long windows
short_window = 2
long_window = 13

#Set names of the windows
short_name = "SMA2"
long_name = "SMA13"

# Generate the short and long moving averages 
signals_df[short_name] = signals_df["SPY Close"].rolling(window=short_window).mean()
signals_df[long_name] = signals_df["SPY Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the short SMA is under the long SMA, and
# where 1 is when the short SMA is higher (or crosses over) the long SMA
signals_df["Signal"][short_window:] = np.where(
    signals_df[short_name][short_window:] > signals_df[long_name][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Buy/Sell"] = signals_df["Signal"].diff()

signals_df.to_csv('signals_2_13.csv') 
signals_df.tail(5)


,SPY Close,SMA2,SMA13,Signal,Buy/Sell
date,,,,,
2020-08-07,334.57,334.450,326.965385,1.0,0.0
2020-08-10,335.57,335.070,327.635385,1.0,0.0
2020-08-11,332.80,334.185,328.392308,1.0,0.0
2020-08-12,337.44,335.120,329.666154,1.0,0.0
2020-08-13,336.83,337.135,330.713077,1.0,0.0
2020-08-14,336.84,336.835,331.918462,1.0,0.0
2020-08-17,337.91,337.375,332.902308,1.0,0.0
2020-08-18,338.64,338.275,334.031538,1.0,0.0
2020-08-19,337.23,337.935,334.855385,1.0,0.0


In [84]:
if signals_df['Buy/Sell'].iloc[-1] == 1.0:
    symbol = "SPY"
    alpaca_functions.buy(symbol)
    
elif signals_df['Buy/Sell'].iloc[-1] == -1.0:
    symbol = "SPY"
    alpaca_functions.sell(symbol)
    
elif signals_df['Buy/Sell'].iloc[-1] == 0.0:
    pass


In [13]:

assets = api.list_assets()
ticker = "SPY"
if ticker in assets:
    print('yes')
else:
    print('no')

no
